In [86]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import seaborn as sns
from pandas.io.json import json_normalize
import folium
import requests
from collections import Counter, defaultdict

### Companies Mongodb:

In [87]:
conn = MongoClient("localhost:27017")
conn.list_database_names()
db = conn.get_database("ironhack")
db.list_collection_names()

['restaurants', 'books', 'companies', 'countries-small']

In [88]:
companies = db.get_collection("companies")
companies.find_one({}).keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

From this dataset we will select all the companies that where founded during and after the economic crisis of 2009. Showing only the following keys:

    - name
    - founded_year (We asume the economic crisis was fully settled in 2009, although LB crashed in sept. 2008)
    - offices (nested array, to find location of the office)

In [89]:
companies.find(
        {'founded_year': {"$gt": 2008}},   
    {"name": 1, 'founded_year': 1, "offices" : 1, "_id" : 0})

In [90]:
df_companies = pd.DataFrame(companies.find(
        {'founded_year': {"$gt": 2008}},   
    {"name": 1, 'founded_year': 1, "offices" : 1, "_id" : 0}))

Next we create a new raw for each office location without loosing its initial index:

In [91]:
df_companies = df_companies.explode('offices')

In [92]:
df_companies.sample(3)

,name,founded_year,offices
422,Leadjini,2009,NaN
336,Foursquare,2009,"{'description': '', 'address1': '568 Broadway'..."
66,Garena,2009,"{'description': '', 'address1': '', 'address2'..."


In [93]:
df_companies.offices.isna().sum()

312

In [94]:
df_companies = df_companies.dropna()

In [95]:
df_companies["city"] = df_companies["offices"].apply(lambda o : o.get("city"))

In [96]:
df_companies.head()

,name,founded_year,offices,city
0,Mokitown,2011,"{'description': None, 'address1': None, 'addre...",None
1,CircleUp,2011,"{'description': '', 'address1': '', 'address2'...",San Francisco
2,PeekYou,2012,"{'description': None, 'address1': None, 'addre...",New York
3,GENWI,2010,"{'description': '', 'address1': '4966 El Camin...",Los Altos
4,headr,2012,"{'description': '', 'address1': 'Ritterstr. 12...",Berlin


 The cities to look for:
 
    - Madrid 
    - New York 
    - San Francisco
    - London
    - Hannover

In [76]:
selected_words = ["Madrid", "New York", "San Francisco", "London", "Hannover"] 

In [77]:
df_companies.city.value_counts()[selected_words]

Madrid            4
New York         39
San Francisco    50
London           27
Hannover          1
Name: city, dtype: int64

The selected city is San Francisco

In [78]:
df_companies =  df_companies[df_companies.city == 'San Francisco']

In [80]:
df_companies["latitude"] = df_companies["offices"].apply(lambda o : o.get("latitude"))

In [82]:
df_companies["longitude"] = df_companies["offices"].apply(lambda o : o.get("longitude"))

In [83]:
df_companies

,name,founded_year,offices,city,latitude,longitude
1,CircleUp,2011,"{'description': '', 'address1': '', 'address2'...",San Francisco,NaN,NaN
11,Widgetbox,2012,"{'description': None, 'address1': None, 'addre...",San Francisco,37.798853,-122.398599
17,YouNoodle,2010,"{'description': '', 'address1': '330 Townsend ...",San Francisco,37.776659,-122.395875
22,Ninite,2009,"{'description': '', 'address1': '955 Bush St.'...",San Francisco,37.789349,-122.412893
28,Macroaxis,2009,"{'description': 'Macroaxis Inc', 'address1': '...",San Francisco,37.789629,-122.399878
33,Supercool School,2009,"{'description': None, 'address1': '1176 South ...",San Francisco,37.754332,-122.416804
62,Advisor,2011,"{'description': 'San Francisco HQ', 'address1'...",San Francisco,37.776399,-122.417716
64,Prezi,2009,"{'description': 'SF Office', 'address1': '735 ...",San Francisco,NaN,NaN
71,Integrate,2010,"{'description': 'San Francisco Office', 'addre...",San Francisco,NaN,NaN
81,Integrate,2010,"{'description': 'San Francisco Office', 'addre...",San Francisco,NaN,NaN


In [84]:
df_companies = df_companies.dropna()

Our final dataframe includes all the cities founded before 2008 in San Francisco with a non-null value for latitude and longitude. 

In [85]:
df_companies

,name,founded_year,offices,city,latitude,longitude
11,Widgetbox,2012,"{'description': None, 'address1': None, 'addre...",San Francisco,37.798853,-122.398599
17,YouNoodle,2010,"{'description': '', 'address1': '330 Townsend ...",San Francisco,37.776659,-122.395875
22,Ninite,2009,"{'description': '', 'address1': '955 Bush St.'...",San Francisco,37.789349,-122.412893
28,Macroaxis,2009,"{'description': 'Macroaxis Inc', 'address1': '...",San Francisco,37.789629,-122.399878
33,Supercool School,2009,"{'description': None, 'address1': '1176 South ...",San Francisco,37.754332,-122.416804
62,Advisor,2011,"{'description': 'San Francisco HQ', 'address1'...",San Francisco,37.776399,-122.417716
103,Formspring,2009,"{'description': 'Formspring', 'address1': '', ...",San Francisco,39.905226,-86.054702
140,Indee,2010,"{'description': 'Headquarters', 'address1': '1...",San Francisco,37.805324,-122.405276
166,Heyzap,2009,"{'description': 'Head Quaters', 'address1': '3...",San Francisco,37.790554,-122.404149
174,Meez,2009,"{'description': '', 'address1': '620 Folsom St...",San Francisco,37.785271,-122.397582
